In [144]:
# Generate skewed random obs
# Generate calibrated forecasts
# Take District 90th percentile forecast and obs value
# Measure what functional it is targeting

In [145]:
import scipy.stats as ss
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import xarray as xr
from scores.continuous import isotonic_fit
from scores.categorical import firm
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [146]:
np.random.seed(100)

In [147]:
TIMESTEPS = 10000

# Create linear variation
fcst = np.arange(0, 4, 0.01)
# broadcast the same thing across 100 timesteps
fcst = fcst * np.array([[1]*len(fcst)]*TIMESTEPS)
# Make the forecast vary with each timestep
fcst = fcst * np.array([np.random.random(TIMESTEPS)]).T#+ np.array([np.random.random(TIMESTEPS)]).T - 0.5
# Make a DataArray to check dims are the right way
fcst = xr.DataArray(data=fcst.T, dims = ["x", "time"], coords={"x": np.arange(0, 400), "time": np.arange(1, TIMESTEPS +1)})

In [148]:
obs = xr.DataArray(data=ss.norm.rvs(loc=fcst.values, scale=1), dims = ["x", "time"], coords={"x": np.arange(0, 400), "time": np.arange(1,TIMESTEPS+1)})

In [149]:
print(fcst.mean(), obs.mean())
print(fcst.median(), obs.median())

<xarray.DataArray ()>
array(0.9928507) <xarray.DataArray ()>
array(0.99234819)
<xarray.DataArray ()>
array(0.73889696) <xarray.DataArray ()>
array(0.91389867)


In [150]:
obs.mean()

<xarray.DataArray ()>
array(0.99234819)

In [151]:
quantileobs = obs.quantile(0.9, dim="x")
quantilefcst = fcst.quantile(0.9, dim="x")

In [152]:
# pd.DataFrame(data = np.array([quantilefcst, quantileobs]), columns=["quantile_fcst", "quantile_obs"])
quantile_timeseries = pd.DataFrame(data={"District forecast": quantilefcst, "District observation": quantileobs})
quantile_timeseries.index.name = "day"
quantile_timeseries.index = quantile_timeseries.index + 1

In [153]:
quantile_timeseries = quantile_timeseries[quantile_timeseries.index < 71]

In [154]:
fig = px.line(quantile_timeseries)
fig.update_layout(
    xaxis_title='Day',
    yaxis_title='Synthetic EHF<sub>sev</sub> district value',
    width=600,
    height=400,
    margin=dict(l=0,r=10,b=50,t=10),
    legend=dict(
        yanchor="bottom",
        y=0.01,
        xanchor="right",
        x=0.99,
        title=None
    )
)
fig.update_xaxes(dtick=10)
fig.show()

In [155]:
# fig.write_image("../figures/synthetic.png", scale=5)

In [156]:
iso_dict = isotonic_fit(quantilefcst, quantileobs, functional="quantile", quantile_level=0.5)

fig = go.Figure()
max_val = np.maximum(quantilefcst.max(), quantileobs.max())
min_val = np.minimum(quantilefcst.min(), quantileobs.min())

fig.add_trace(go.Scatter(
    x=quantilefcst, 
    y=quantileobs, 
    mode='markers', 
    line=dict(color="rgba(24, 20, 247, 0.2)"), 
    showlegend=False,
    ))
fig.add_trace(go.Scatter(
    x=iso_dict["fcst_sorted"], 
    y=iso_dict["regression_values"], 
    line=dict(color="red"),
    showlegend=False,
    ))
fig.add_trace(go.Scatter(
    x=[min_val, max_val], 
    y=[min_val, max_val], 
    name="ref", 
    line=dict(color="black"), 
    mode="lines", 
    showlegend=False,
    ))

fig.update_layout(
width=400,
height=600,
margin=go.layout.Margin(
    l=0, #left margin
    r=20, #right margin
    b=0, #bottom margin
    t=20, #top margin
),
)
fig.show()

# Create subplot

In [157]:
fig = make_subplots(rows=2, cols=1, subplot_titles=("<b>(a)</b>", "<b>(b)</b>"), vertical_spacing=0.13)
fig.update_annotations(font_size=12, xshift=-180, xanchor="left")

# Subplot 1
fig_timeseries = px.line(quantile_timeseries)
fig1_traces=[]
for trace in range(len(fig_timeseries["data"])):
    fig1_traces.append(fig_timeseries["data"][trace])

for traces in fig1_traces:
    fig.append_trace(traces, row=1, col=1)



fig.update_layout(
    width=400,
    height=600,
    margin=go.layout.Margin(
        l=0, #left margin
        r=10, #right margin
        b=20, #bottom margin
        t=20, #top margin
    ),
    legend=dict(
        yanchor="bottom",
        y=0.565,
        xanchor="right",
        x=0.999
    )
)

iso_dict = isotonic_fit(quantilefcst, quantileobs, functional="quantile", quantile_level=0.5)

max_val = np.maximum(quantilefcst.max(), quantileobs.max())
min_val = np.minimum(quantilefcst.min(), quantileobs.min())

fig.add_trace(go.Scatter(
    x=quantilefcst, 
    y=quantileobs, 
    mode='markers', 
    line=dict(color="rgba(24, 20, 247, 0.1)"), 
    showlegend=False,
    ), row=2, col=1)
fig.add_trace(go.Scatter(
    x=iso_dict["fcst_sorted"], 
    y=iso_dict["regression_values"], 
    line=dict(color="red"),
    showlegend=False,
    ), row=2, col=1)
fig.add_trace(go.Scatter(
    x=[min_val, max_val], 
    y=[min_val, max_val], 
    name="ref", 
    line=dict(color="black"), 
    mode="lines", 
    showlegend=False,
    ), row=2, col=1)

fig.update_yaxes(title_text="EHF<sub>SEV</sub>", row=1, col=1)
fig.update_yaxes(title_text="Observed EHF<sub>SEV</sub>", row=2, col=1)
fig.update_xaxes(title_text="Day", row=1, col=1, dtick=10)
fig.update_xaxes(title_text="Forecast EHF<sub>SEV</sub>", row=2, col=1, dtick=1)
fig.show()

In [158]:
fig.write_image("../figures/synthetic_subplot.png", scale=5)

# Now do a test - train split to evaluate the impact of recalibration

In [ ]:
TRAINING_COUNT = 100
quantilefcst_train = quantilefcst.where(quantilefcst.time <= TRAINING_COUNT, drop=True)
quantileobs_train = quantileobs.where(quantileobs.time <= TRAINING_COUNT, drop=True)

quantilefcst_test = quantilefcst.where(quantilefcst.time > TRAINING_COUNT, drop=True)
quantileobs_test = quantileobs.where(quantileobs.time > TRAINING_COUNT, drop=True)

iso_dict = isotonic_fit(quantilefcst_train, quantileobs_train, functional="quantile", quantile_level=0.5)

In [ ]:
quantilefcst_recal = quantilefcst_test.copy()
quantilefcst_recal.values = iso_dict["regression_func"](quantilefcst_test)

# Fill extremes
quantilefcst_recal = quantilefcst_recal.where(quantilefcst_test < quantilefcst_train.max().item(), 3.5)
quantilefcst_recal = quantilefcst_recal.where(quantilefcst_test > quantilefcst_train.min().item(), -1)

In [ ]:
firm(quantilefcst_test, quantileobs_test, 0.5, [1, 3], [2, 1], threshold_assignment="upper")

<xarray.Dataset>
Dimensions:                ()
Coordinates:
    quantile               float64 0.9
Data variables:
    firm_score             float64 0.341
    overforecast_penalty   float64 0.0
    underforecast_penalty  float64 0.341

In [ ]:
firm(quantilefcst_recal, quantileobs_test, 0.5, [1, 3], [2, 1], threshold_assignment="upper")

<xarray.Dataset>
Dimensions:                ()
Coordinates:
    quantile               float64 0.9
Data variables:
    firm_score             float64 0.0701
    overforecast_penalty   float64 0.03025
    underforecast_penalty  float64 0.03985

In [ ]:
firm_recal_sample_list = []
firm_test_sample_list = []

for i in np.arange(0, 1000):
    random_idx = np.random.randint(101, 10001, 1000)
    quantilefcst_recal_sample = quantilefcst_recal.sel(time=random_idx)
    quantilefcst_test_sample = quantilefcst_test.sel(time=random_idx)
    obs_sample = quantileobs_test.sel(time=random_idx)

    firm_recal_sample = firm(quantilefcst_recal_sample, obs_sample, 0.5, [1, 3], [2, 1], threshold_assignment="upper")
    firm_test_sample = firm(quantilefcst_test_sample, obs_sample, 0.5, [1, 3], [2, 1], threshold_assignment="upper")

    firm_recal_sample_list.append(firm_recal_sample.firm_score.item())
    firm_test_sample_list.append(firm_test_sample.firm_score.item())

In [ ]:
# mean FIRM score with lower/upper CIs
np.mean(firm_recal_sample_list), np.quantile(firm_recal_sample_list, 0.05), np.quantile(firm_recal_sample_list, 0.95)

(0.070029, 0.0605, 0.0805)

In [ ]:
# mean FIRM score with lower/upper CIs
np.mean(firm_test_sample_list), np.quantile(firm_test_sample_list, 0.05), np.quantile(firm_test_sample_list, 0.95)

(0.340154, 0.319, 0.3615)